In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [14]:
# There was something Jocelyn did to modify the image sizes without modifying the dtype
size = 14
root_dir = "../data/" + str(size) + "x" + str(size)

train_data = datasets.MNIST(
    root = root_dir,
    train = True,                         
    transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize(size),
                    transforms.Normalize(0, 1)
                ]), 
    download = False,            
)
test_data = datasets.MNIST(
    root = root_dir, 
    train = False, 
    transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize(size),
                    transforms.Normalize(0, 1)
                ]),
    download = False,
)

In [16]:
train_data[0][0].shape

torch.Size([1, 28, 28])

<h1>Using Pytorch's Data loaders to feed data into CNN</h1>

In [17]:
# No change required
loaders = {
    'train' : torch.utils.data.DataLoader(train_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
    
    'test'  : torch.utils.data.DataLoader(test_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
}
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7ffc72bbb2b0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7ffc75fbb7f0>}

<h1>Defining the CNN</h1>

In [19]:
# Requires working on 
class CNN(nn.Module):
    def __init__(self, size):
        super(CNN, self).__init__()
        '''
        Goes through two layers of 2D convolutions, each followed by a 2x2 max pooling
        
        Dimensions:
        1.) Input Shape:          [batch_size, 1, 14, 14]
        
        2.) After conv:           [batch_size, 4, 12, 12]
            After max pooling:    [batch_size, 4, 6, 6]
        
        3.) After conv:           [batch_size, 8, 4, 4]
            After max pooling:    [batch_size, 8, 2, 2]
            
        4.) After flattening:     [batch_size, 8 * 2 * 2]
        '''
        if size == 28:
            self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=0,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
            )
            self.conv2 = nn.Sequential(         
                nn.Conv2d(16, 32, 5, 1, 0),     
                nn.ReLU(),                      
                nn.MaxPool2d(2),                
            )
            # fully connected layer, output 10 classes
            self.out = nn.Linear(32 * 4 * 4, 10)
            pass
        
        elif size == 14:
            self.conv1 = nn.Sequential(         
                nn.Conv2d(
                    in_channels=1,              
                    out_channels=4,            
                    kernel_size=3,              
                    stride=1,                   
                    padding=0,                  
                ),                              
                nn.ReLU(),                      
                nn.MaxPool2d(kernel_size=2),    
            )
            self.conv2 = nn.Sequential(         
                nn.Conv2d(4, 8, 3, 1, 0),     
                nn.ReLU(),                      
                nn.MaxPool2d(2),                
            )
            # fully connected layer, output 10 classes
            # compute Linear Layer input size
            self.out = nn.Linear(8 * 2 * 2, 10)
            pass
        
        elif size == 7:
            self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=4,            
                kernel_size=2,              
                stride=1,                   
                padding=0,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
            )
            self.conv2 = nn.Sequential(         
                nn.Conv2d(4, 8, 2, 1, 0),     
                nn.ReLU(),                      
                nn.MaxPool2d(2),                
            )
            # fully connected layer, output 10 classes
            self.out = nn.Linear(8 * 1 * 1, 10)
            pass
        
        elif size == 4:
            self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=4,            
                kernel_size=3,              
                stride=1,                   
                padding=1,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
            )
            self.conv2 = nn.Sequential(         
                nn.Conv2d(4, 8, 3, 1, 1),     
                nn.ReLU(),                      
                nn.MaxPool2d(2),                
            )
            # fully connected layer, output 10 classes
            self.out = nn.Linear(8 * 1 * 1, 10)
            pass
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 16 * 8 * 8)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    # return x for visualization
    
    
    def flatten_input(in_size, ker_size, padding, stride):
        pass

In [21]:
cnn = CNN(14)
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=32, out_features=10, bias=True)
)


In [ ]:
loss_func = nn.CrossEntropyLoss(); print(loss_func)   
optimizer = torch.optim.Adam(cnn.parameters(), lr= 1e-5); print(optimizer)

<h1>Training the CNN</h1>
<div>Error source may be that the dataset resized_28 doesn't have 'labels', just 'images' ---> (for i, (images, labels))</div>

In [ ]:
num_epochs = 50
train_acc_data = []
loss_data = []

def train(num_epochs, cnn, loaders):
    cnn.train()
        
    # Train the model
    total_step = len(loaders['train'])
        
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y
            output = cnn(b_x)[0]               
            loss = loss_func(output, b_y)
            
            # measure accuracy and record loss
            train_output, last_layer = cnn(images)
            pred_y = torch.max(train_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()  
#             output = cnn(images)
#             correct += (output == labels).float().sum()
#             accuracy = 100 * correct / len(trainset)
            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(), accuracy))
                train_acc_data.append(accuracy)
                loss_data.append(loss)
            pass
        
        pass
    
    
    pass
    
train(num_epochs, cnn, loaders)

In [ ]:
num_epochs = 10
test_acc_data = []

def test():
    cnn.eval()
        
    # Train the model
    correct = 0
    total = 0
    for images, labels in loaders['test']:
        test_output, last_layer = cnn(images)
        pred_y = torch.max(test_output, 1)[1].data.squeeze()
        accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
        test_acc_data.append(accuracy)
        print("Accuracy: {:.4f}".format(accuracy))
        
#     print('Test Accuracy: %.2f' % accuracy)
    
test()

In [ ]:
# for i in range(len(acc_data)):
#     if acc_data[i] == 0.71:
#         print("fount at ", i)

# acc_data[600-1]

In [ ]:
f, axarr = plt.subplots(1,3, figsize=(20, 5))
axarr[0].plot(train_acc_data)
axarr[0].set_title("Train Accuracy")
axarr[1].plot(test_acc_data)
axarr[1].set_title("Test Accuracy")
axarr[2].plot(loss_data)
axarr[2].set_title("Loss")

In [ ]:
cnn.train()
len(loaders['train'])

In [ ]:
type(train_data)

In [ ]:
type(resized_28)

In [ ]:
# for i, image in enumerate(loaders['train']):
#     print(i, image.shape)

In [ ]:
fig = plt.figure()
fig.set_figwidth(8)
fig.set_figheight(30)
plt.imshow(test_data[0][0][0])